In [1]:
import os
import json
import zipfile
from huggingface_hub import hf_hub_download
from tqdm.auto import tqdm
from io import TextIOWrapper
import pandas as pd
from io import BytesIO

### Загрузка и подготовка данных о Telegram-каналах

Эта ячейка выполняет следующие задачи:

1. **Скачивает** архив `all.channels.csv.zip` из датасета на Hugging Face  
   (`ajtkulov/telegram-ru`)

2. **Распаковывает** его в локальную папку `data/posts/ajtkulov/meta`

3. **Находит** внутри распакованный CSV-файл (независимо от того, как он называется)

4. **Читает** CSV с максимально устойчивой обработкой проблем:
   - пробует несколько популярных кодировок (utf-8-sig → cp1251 → utf-8 и др.)
   - пропускает битые строки
   - заменяет нераспознанные символы
   - использует разделитель табуляцию (`\t`)

5. **Переименовывает** столбцы в понятные имена, если их количество совпадает с ожидаемым:
   - `link`, `name`, `description`, `category`, `message_id`

6. **Выводит** базовую статистику:
   - общее количество каналов
   - список столбцов
   - первые 10 строк
   - распределение по категориям (топ-150)
   - список всех уникальных категорий

7. **Сохраняет** таблицу с уникальными категориями в файл `unique_categories.csv`  
   (удобно для дальнейшего анализа и маппинга)

**Результат работы ячейки** — подготовленный pandas-датафрейм `df_channels` с мета-информацией о тысячах Telegram-каналов, готовый к дальнейшей фильтрации, выборке или использованию в пайплайне сбора постов.

In [2]:
# Параметры
repo_id = "ajtkulov/telegram-ru"
meta_zip_filename = "meta/all.channels.csv.zip"          # путь в репозитории
output_meta_dir = "data/posts/ajtkulov/meta"             # куда сохраняем

os.makedirs(output_meta_dir, exist_ok=True)

# Шаг 1: Скачивание ZIP-файла
try:
    zip_path = hf_hub_download(
        repo_id=repo_id,
        filename=meta_zip_filename,
        repo_type="dataset"
    )
    print(f"ZIP-файл скачан: {zip_path}")
except Exception as e:
    print(f"Ошибка скачивания: {e}")
    raise

# Шаг 2: Распаковка в нужную папку
print("Распаковка в папку:", output_meta_dir)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    inner_files = zip_ref.namelist()
    print(f"Файлы внутри ZIP: {inner_files}")
    
    if not inner_files:
        raise ValueError("ZIP-файл пустой")
    
    zip_ref.extractall(output_meta_dir)
    print("Распаковка завершена")

# Шаг 3: Поиск распакованного CSV-файла
csv_path = None
for file in os.listdir(output_meta_dir):
    if file.lower().endswith('.csv'):
        csv_path = os.path.join(output_meta_dir, file)
        break

if not csv_path:
    raise FileNotFoundError("CSV-файл не найден после распаковки. Проверьте папку.")

print(f"\nЧтение CSV из файла: {csv_path}")

# Шаг 4: Чтение с обработкой всех типичных проблем
encodings = ['utf-8-sig', 'cp1251', 'utf-8', 'latin1', 'iso-8859-1']
df_channels = None

for encoding in encodings:
    try:
        df_channels = pd.read_csv(
            csv_path,
            encoding=encoding,
            sep='\t',                  # табуляция (как у вас)
            on_bad_lines='skip',       # пропуск битых строк
            low_memory=False,
            encoding_errors='replace'  # замена кракозябр
        )
        print(f"\nУСПЕХ! Файл прочитан с кодировкой: {encoding}")
        break
    except Exception as e:
        print(f"Попытка с {encoding} провалилась: {str(e)}")

if df_channels is None:
    raise ValueError("Не удалось прочитать CSV ни одной кодировкой. Возможно файл повреждён.")

# ← Добавляем названия колонок
column_names = ["link", "name", "description", "category", "message_id"]
if len(df_channels.columns) == len(column_names):
    df_channels.columns = column_names
    print("\nКолонки успешно переименованы в: link, name, description, category, message_id")
else:
    print(f"\nВнимание! Количество столбцов ({len(df_channels.columns)}) не равно ожидаемому ({len(column_names)}).")
    print("Колонки НЕ переименованы. Текущие:", df_channels.columns.tolist())


# Общая статистика
print(f"Количество каналов: {len(df_channels):,}")
print(f"Количество столбцов: {len(df_channels.columns)}")
print("Столбцы:", df_channels.columns.tolist())

# Первые 10 строк
print("\nПервые 10 каналов:")
display(df_channels.head(10))

# Распределение категорий
if 'category' in df_channels.columns:
    print("\nРаспределение по category (топ-15):")
    print(df_channels['category'].value_counts().head(150))
else:
    print("\nКолонка 'category' не найдена")

if 'category' in df_channels.columns:
    unique_categories = df_channels['category'].dropna().unique()
    unique_df = pd.DataFrame(unique_categories, columns=['category'])
    
    unique_csv_path = os.path.join(output_meta_dir, "unique_categories.csv")
    unique_df.to_csv(unique_csv_path, index=False, encoding='utf-8-sig')
    
    print(f"\nУникальные категории сохранены в файл: {unique_csv_path}")
    print(f"Количество уникальных категорий: {len(unique_categories)}")
    print("Первые 20 уникальных категорий:")
    print(unique_df.head(20))
else:
    print("\nНе удалось сохранить категории — колонка 'category' отсутствует")

print("CSV лежит здесь:", csv_path)

ZIP-файл скачан: C:\Users\Admin\.cache\huggingface\hub\datasets--ajtkulov--telegram-ru\snapshots\e7c2668f8ffe8d7b9725d4639d3f9e96a25a58b4\meta\all.channels.csv.zip
Распаковка в папку: data/posts/ajtkulov/meta
Файлы внутри ZIP: ['all.channels.csv']
Распаковка завершена

Чтение CSV из файла: data/posts/ajtkulov/meta\all.channels.csv

УСПЕХ! Файл прочитан с кодировкой: utf-8-sig

Колонки успешно переименованы в: link, name, description, category, message_id
Количество каналов: 380,467
Количество столбцов: 5
Столбцы: ['link', 'name', 'description', 'category', 'message_id']

Первые 10 каналов:


,link,name,description,category,message_id
0,https://tgstat.ru/channel/@premium,Telegram Premium,Telegram Premium – a subscription that unlocks...,Telegram,7980100.0
1,https://tgstat.ru/channel/sOj9iDAtUkMyYWQy,Топор Live,"Нейтрально, без пропаганды. Топор Live с быстр...",Новости,4388359.0
2,https://tgstat.ru/channel/@wewantyoutodothejob,WeWantYou,Канал для поиска исполнителей для разных задач...,Другое,4158678.0
3,https://tgstat.ru/channel/@leoday,Леонардо Дайвинчик,Бот знакомств @leomatchbot,Юмор и развлечение,4057819.0
4,https://tgstat.ru/channel/@novosti_efir,Прямой Эфир • Новости,️Все самое важное в одном канале. Новости Росс...,Новости,3886208.0
5,https://tgstat.ru/channel/@novosti_voinaa,СМИ Россия не Москва,Эруктации информпространства России и ее Окраин.,Новости,3368394.0
6,https://tgstat.ru/channel/@rian_ru,РИА Новости,Главные Новости РИА t.me/rian_ru,Новости,3220976.0
7,https://tgstat.ru/channel/@invest_zonaa,INVEST ZONE,"Привет! Я Руслан, с 2017 года торгую рынок кри...",Криптовалюты,3057506.0
8,https://tgstat.ru/channel/@mash,Mash,"Прислать новость, фото, видео, аудио, бересту:...",Новости,2827677.0
9,https://tgstat.ru/channel/@crypto_drop_stukach,Дропы от Стукача,"Все о крипто раздачах, прибыльных темах и абуз...",Шок-конент,2636192.0



Распределение по category (топ-15):
category
Новости                                                               25909
Блоги                                                                 25599
Другое                                                                20693
Мода и красота                                                        17673
Психология                                                            13740
                                                                      ...  
Политика|||Регион|||Свердловская область                                 19
Политика|||Регион|||Самарская область                                    19
Путешествия|||Регион|||Приморский край                                   18
Новости|||Регион|||Кабардино-Балкарская Республика|||Новости и СМИ       18
Политика|||Регион|||Пермский край                                        18
Name: count, Length: 150, dtype: int64

Уникальные категории сохранены в файл: data/posts/ajtkulov/meta\unique_categor

### Фильтрация категорий Telegram-каналов (удаление нежелательных)

Эта ячейка очищает список уникальных категорий, полученный на предыдущем шаге, от нежелательных, слишком общих или проблемных тематик.

**Что делает ячейка:**

1. Читает файл `unique_categories.csv` (созданный ранее)
2. Применяет жёсткую фильтрацию по двум уровням:
   - **Точное совпадение** с запрещённым списком (`FORBIDDEN_PATTERNS`):
     - Новости, Политика, Шок-контент, Darknet, Для взрослых, Эротика, Право, Религия, Инстаграм, Другое, Telegram и др.
   - **Содержит подстроку** из списка `FORBIDDEN_SUBSTRINGS` (даже частичное вхождение):
     - Новости, Политика, Шок, Darknet
3. Удаляет все строки, попавшие под любой из этих критериев
4. Убирает возможные дубликаты и пустые значения
5. Сохраняет очищенный список в новый файл:  
   `data/posts/ajtkulov/meta/filtered_categories.csv`

**Цель фильтрации**  
Оставить только потенциально полезные, нишевые категории, которые могут быть связаны с товарами, услугами, хобби, техникой, красотой, спортом и т.п.  
Исключаются:  
- новостные / политические / шок-контент каналы  
- слишком общие категории  
- взрослый контент и сомнительные тематики

**Результат**  
Чистый, компактный список категорий, который удобно использовать для:
- выборки каналов по тематике
- последующего сбора постов
- маппинга на категории товаров

После выполнения ячейки рекомендуется посмотреть на первые 20–30 строк результата и при необходимости дополнить список запрещённых паттернов.

In [3]:
# Ячейка: Фильтрация категорий (убираем ненужные, включая подстроки)

import pandas as pd
import os

# Путь к файлу с уникальными категориями
input_csv = 'data/posts/ajtkulov/meta/unique_categories.csv'

# Путь для сохранения отфильтрованного файла
output_csv = 'data/posts/ajtkulov/meta/filtered_categories.csv'

print("==================================================")
print(f"Чтение файла: {input_csv}")

# Читаем файл (учитывая BOM и кодировку)
df = pd.read_csv(input_csv, encoding='utf-8-sig')

print(f"Всего уникальных категорий в исходном файле: {len(df):,}")
print("Первые 10 категорий:")
print(df.head(10))

# Список запрещённых категорий и подстрок (расширенный)
FORBIDDEN_PATTERNS = [
    'Новости', 'Политика', 'Шок-контент', 'Шок-конент', 'Darknet',
    'Для взрослых', 'Эротика', 'Право', 'Религия', 'Инстаграм',
    'Другое', 'Telegram',  # слишком общие
]

# Дополнительно: любые категории, содержащие эти слова (даже с |||)
FORBIDDEN_SUBSTRINGS = ['Новости', 'Политика', 'Шок', 'Darknet']

print("\nЗапрещённые категории / подстроки (удаляем их):")
for p in FORBIDDEN_PATTERNS + FORBIDDEN_SUBSTRINGS:
    print(f" - {p}")

# Фильтрация:
# 1. Точное совпадение с запрещённым списком
# 2. Содержит любую запрещённую подстроку
mask = (
    df['category'].isin(FORBIDDEN_PATTERNS) |
    df['category'].str.contains('|'.join(FORBIDDEN_SUBSTRINGS), na=False, regex=True)
)

filtered_df = df[~mask].copy()

# Убираем дубликаты и NaN
filtered_df = filtered_df.dropna(subset=['category']).drop_duplicates(subset=['category'])

print(f"\nПосле фильтрации осталось категорий: {len(filtered_df):,}")

print("\nПервые 20 оставшихся категорий:")
print(filtered_df.head(20))

# Сохранение отфильтрованного файла
filtered_df.to_csv(output_csv, index=False, encoding='utf-8')

print(f"\nОтфильтрованный файл сохранён: {output_csv}")
print("Теперь в нём только нишевые / полезные категории")

Чтение файла: data/posts/ajtkulov/meta/unique_categories.csv
Всего уникальных категорий в исходном файле: 1,748
Первые 10 категорий:
                                    category
0                                   Telegram
1                                    Новости
2                                     Другое
3                         Юмор и развлечение
4                               Криптовалюты
5                                 Шок-конент
6  Новости|||Регион|||Москва|||Новости и СМИ
7                                   Политика
8                                  Экономика
9                                 Технологии

Запрещённые категории / подстроки (удаляем их):
 - Новости
 - Политика
 - Шок-контент
 - Шок-конент
 - Darknet
 - Для взрослых
 - Эротика
 - Право
 - Религия
 - Инстаграм
 - Другое
 - Telegram
 - Новости
 - Политика
 - Шок
 - Darknet

После фильтрации осталось категорий: 1,417

Первые 20 оставшихся категорий:
              category
3   Юмор и развлечение
4         Крип

### Очистка первого столбца мета-данных (нормализация имён каналов)

Эта ячейка приводит первый столбец исходного файла `all.channels.csv` к единому, чистому и удобному формату — остаётся **только username канала** (например, `channelname` вместо `https://tgstat.ru/channel/@channelname` или `@channelname`).

**Зачем это нужно:**

В основном датасете каналы представлены в разных форматах:  
- полные ссылки `https://tgstat.ru/channel/@username`  
- ссылки с @ в начале `@username`  
- иногда просто username без префиксов  

Такой разнобой мешает:  
- корректно искать каналы по имени  
- фильтровать по категориям  
- сопоставлять с другими источниками (например, с постами, собранными через Telethon)  
- формировать единообразные ссылки `t.me/username`  

Поэтому проводится **нормализация** — из всех вариантов остаётся только чистое имя канала без префиксов, @ и лишних символов. Это критически важно для дальнейшей работы: поиска по категориям, выборки каналов, сбора постов и построения связей «категория → канал → пост → товар».

**Что делает ячейка шаг за шагом:**

1. Читает файл `all.channels.csv`  
   - Сначала пробует `utf-8-sig` + разделитель `\t`  
   - При ошибке — fallback на `cp1251`

2. Если первый столбец без имени и содержит `https://tgstat.ru` → переименовывает его в `link`

3. Применяет функцию очистки:
   - Удаляет префикс `https://tgstat.ru/channel/`
   - Убирает ведущий `@`
   - Удаляет лишние пробелы
   - Обрабатывает пропущенные значения

4. Показывает первые 10 строк для визуального контроля

5. Сохраняет результат в новый файл:  
   `data/posts/ajtkulov/meta/all_channels_clean.csv`

**Результат**  
Файл с нормализованными именами каналов, готовый к использованию в следующих шагах:  
- фильтрация по категориям  
- выбор целевых каналов  
- сбор постов через Telegram API  
- дальнейшее сопоставление с товарами

In [4]:
# Ячейка: Очистка первого столбца (оставляем только имя канала)

import os
import pandas as pd

# Путь к исходному файлу
input_file = 'data/posts/ajtkulov/meta/all.channels.csv'

# Путь для сохранения очищенного файла
output_file = 'data/posts/ajtkulov/meta/all_channels_clean.csv'

print("==================================================")
print(f"Чтение файла: {input_file}")

# Читаем CSV (учитываем возможные проблемы с кодировкой и разделителями)
try:
    df = pd.read_csv(
        input_file,
        encoding='utf-8-sig',
        sep='\t',                  # если табуляция
        on_bad_lines='skip',
        low_memory=False
    )
    print(f"Файл успешно прочитан. Строк: {len(df):,}")
except Exception as e:
    print(f"Ошибка чтения: {e}")
    # Попробуем другую кодировку
    df = pd.read_csv(
        input_file,
        encoding='cp1251',
        sep='\t',
        on_bad_lines='skip',
        low_memory=False
    )
    print("Успешно прочитано с cp1251")

# Проверяем, есть ли первый столбец (по умолчанию он без имени — берём по индексу)
if df.columns[0].startswith('https://tgstat.ru'):
    print("Первый столбец не имеет имени — переименовываем в 'link'")
    df = df.rename(columns={df.columns[0]: 'link'})

# Очистка первого столбца
def clean_channel_link(link):
    if pd.isna(link):
        return link
    link = str(link).strip()
    # Убираем префикс https://tgstat.ru/channel/
    if link.startswith('https://tgstat.ru/channel/'):
        link = link.replace('https://tgstat.ru/channel/', '')
    # Убираем @ в начале, если остался
    if link.startswith('@'):
        link = link[1:]
    return link

print("\nОчистка первого столбца...")
df['link'] = df['link'].apply(clean_channel_link)

# Показываем результат
print("\nПервые 10 строк после очистки:")
display(df.head(10))

# Сохранение нового файла
df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\nОчищенный файл сохранён: {output_file}")
print(f"Первый столбец теперь содержит только имена каналов (без https и @)")

Чтение файла: data/posts/ajtkulov/meta/all.channels.csv
Файл успешно прочитан. Строк: 380,467
Первый столбец не имеет имени — переименовываем в 'link'

Очистка первого столбца...

Первые 10 строк после очистки:


,link,Топор 18+,Самый популярный русскоязычный Telegram канал.,Шок-конент,8179938
0,premium,Telegram Premium,Telegram Premium – a subscription that unlocks...,Telegram,7980100.0
1,sOj9iDAtUkMyYWQy,Топор Live,"Нейтрально, без пропаганды. Топор Live с быстр...",Новости,4388359.0
2,wewantyoutodothejob,WeWantYou,Канал для поиска исполнителей для разных задач...,Другое,4158678.0
3,leoday,Леонардо Дайвинчик,Бот знакомств @leomatchbot,Юмор и развлечение,4057819.0
4,novosti_efir,Прямой Эфир • Новости,️Все самое важное в одном канале. Новости Росс...,Новости,3886208.0
5,novosti_voinaa,СМИ Россия не Москва,Эруктации информпространства России и ее Окраин.,Новости,3368394.0
6,rian_ru,РИА Новости,Главные Новости РИА t.me/rian_ru,Новости,3220976.0
7,invest_zonaa,INVEST ZONE,"Привет! Я Руслан, с 2017 года торгую рынок кри...",Криптовалюты,3057506.0
8,mash,Mash,"Прислать новость, фото, видео, аудио, бересту:...",Новости,2827677.0
9,crypto_drop_stukach,Дропы от Стукача,"Все о крипто раздачах, прибыльных темах и абуз...",Шок-конент,2636192.0



Очищенный файл сохранён: data/posts/ajtkulov/meta/all_channels_clean.csv
Первый столбец теперь содержит только имена каналов (без https и @)


### Подготовка данных и настроек для выборки постов по целевым категориям

Эта ячейка — **стартовый блок** большого пайплайна по отбору и скачиванию постов из Telegram-каналов.  
Она выполняет всю предварительную настройку и проверку, чтобы дальше можно было безопасно и осмысленно работать с огромным датасетом `ajtkulov/telegram-ru`.

**Что именно делает ячейка:**

1. **Определяет ключевые пути и настройки** (всё в одном месте для удобства изменения):
   - репозиторий на Hugging Face
   - папки для сохранения результатов и кэша
   - файлы прогресса, статистики и временных результатов
   - пути к уже подготовленным мета-файлам (`all_channels_clean.csv` и `filtered_categories.csv`)

2. **Проверяет наличие критически важных файлов**  
   Если хотя бы один из них отсутствует — сразу завершает работу с понятным сообщением об ошибке.

3. **Загружает очищенный список каналов** (`all_channels_clean.csv`)  
   → содержит чистые usernames каналов (уже нормализованные на предыдущем шаге)

4. **Загружает отфильтрованный список разрешённых категорий** (`filtered_categories.csv`)  
   → только те категории, которые мы решили оставить после очистки от новостей, политики, шок-контента и т.п.

5. **Создаёт словарь сопоставления** `channel → category`  
   → позволяет быстро узнать, к какой категории относится любой канал  
   → используется дальше для фильтрации постов именно по нужным тематикам

6. **Формирует множество разрешённых категорий** (`allowed_categories`)  
   → будет использоваться как фильтр при обработке ZIP-архивов с постами

7. **Выводит диагностическую информацию** для контроля:
   - сколько всего каналов и категорий загружено
   - первые 10 разрешённых категорий (чтобы сразу увидеть, что фильтр сработал корректно)
   - предупреждения, если структура файлов не соответствует ожидаемой

**Зачем это важно**

Без этой ячейки дальнейшая работа невозможна или приведёт к ошибкам:  
- мы будем обрабатывать ненужные категории (новости, политика, эротика и т.д.)  
- не сможем связать посты с категориями товаров  
- потеряем контроль над тем, какие каналы и посты мы вообще скачиваем  

**Результат выполнения**  
Готовые к использованию структуры данных:  
- `all_channels_clean` — DataFrame со всеми каналами  
- `channel_to_category` — словарь channel → категория  
- `allowed_categories` — множество только нужных категорий  

Дальше код будет скачивать ZIP-архивы из датасета, фильтровать посты именно по этим категориям и сохранять отобранные посты для последующей генерации/разметки пар «пост — товар».

In [ ]:
import os
import json
import zipfile
from huggingface_hub import hf_hub_download
from tqdm.auto import tqdm
from io import TextIOWrapper
import pandas as pd
import random

# ==============================================
# НАСТРОЙКИ (изменяйте здесь)
# ==============================================

repo_id = "ajtkulov/telegram-ru"

output_dir = "data/posts/ajtkulov/selected"               # куда сохранять финальный результат
cache_dir = os.path.join(output_dir, "cache")             # временный кэш для ZIP
progress_file = os.path.join(output_dir, "progress.json") # какие ZIP уже обработаны
stats_file = os.path.join(output_dir, "channel_top_posts.json")  # топ-посты по каналам
selected_posts_file = os.path.join(output_dir, "selected_posts_temp.jsonl")

os.makedirs(output_dir, exist_ok=True)
os.makedirs(cache_dir, exist_ok=True)

# Пути к файлам с категориями
channels_file = "data/posts/ajtkulov/meta/all_channels_clean.csv"
categories_file = "data/posts/ajtkulov/meta/filtered_categories.csv"

# Проверка существования файлов
print("Загрузка информации о каналах и категориях...")
if not os.path.exists(channels_file):
    print(f"ОШИБКА: Файл не найден: {channels_file}")
    print("Убедитесь, что файл all_channels_clean.csv находится в папке data/posts/ajtkulov/")
    exit()

if not os.path.exists(categories_file):
    print(f"ОШИБКА: Файл не найден: {categories_file}")
    print("Убедитесь, что файл filtered_categories.csv находится в папке data/posts/ajtkulov/")
    exit()

# Загружаем данные о каналах и категориях
all_channels_clean = pd.read_csv(channels_file)
filtered_categories = pd.read_csv(categories_file)

print(f"Загружено каналов: {len(all_channels_clean)}")
print(f"Загружено категорий для фильтрации: {len(filtered_categories)}")

# Создаем словарь channel -> category
channel_to_category = {}
if 'category' in all_channels_clean.columns and 'channel' in all_channels_clean.columns:
    for _, row in all_channels_clean.iterrows():
        channel_to_category[row['channel']] = row['category']
    print(f"Создан словарь категорий для {len(channel_to_category)} каналов")
else:
    print("ОШИБКА: В файле all_channels_clean.csv должны быть колонки 'channel' и 'category'")
    print(f"Найдены колонки: {list(all_channels_clean.columns)}")
    exit()

# Создаем множество разрешенных категорий
if 'category' in filtered_categories.columns:
    allowed_categories = set(filtered_categories['category'].dropna().astype(str).tolist())
    print(f"Разрешено {len(allowed_categories)} категорий")
    print("Первые 10 разрешенных категорий:")
    for i, cat in enumerate(list(allowed_categories)[:10]):
        print(f"  {i+1}. {cat}")
else:
    print("ОШИБКА: В файле filtered_categories.csv должна быть колонка 'category'")
    print(f"Найдены колонки: {list(filtered_categories.columns)}")
    exit()

Загрузка информации о каналах и категориях...
Загружено каналов: 380467
Загружено категорий для фильтрации: 1417
ОШИБКА: В файле all_channels_clean.csv должны быть колонки 'channel' и 'category'
Найдены колонки: ['link', 'Топор 18+', 'Самый популярный русскоязычный Telegram канал.', 'Шок-конент', '8179938']
Разрешено 1417 категорий
Первые 10 разрешенных категорий:
  1. Природа|||Регион|||Республика Башкортостан
  2. Регион|||Сахалинская область|||Спорт
  3. Еда и кулинария|||Регион|||Нижегородская область
  4. Продажи|||Регион|||Чеченская Республика
  5. Регион|||Краснодарский край|||Путешествия
  6. Интерьер и строительство|||Регион|||Рязанская область
  7. Другое|||Регион|||Ханты-Мансийский автономный округ - Югра
  8. Право|||Регион|||Республика Марий Эл
  9. Продажи|||Регион|||Тамбовская область
  10. Бизнес|||Регион|||Самарская область|||Бизнес и стартапы


: 

### Основной пайплайн: скачивание, фильтрация и отбор качественных постов из Telegram-каналов

Это **главная рабочая ячейка** всего проекта по подготовке датасета постов.  
Она скачивает огромный датасет `ajtkulov/telegram-ru` (230 ZIP-архивов), фильтрует посты по категориям каналов, длине текста и стратегии отбора, а затем сохраняет только лучшие / средние по популярности посты в удобных форматах.

**Ключевые параметры (настраиваются в начале):**

- `MAX_POSTS_PER_CHANNEL = 5` — максимум постов с одного канала  
- `MIN_TEXT_LEN = 500` / `MAX_TEXT_LEN = 1200` — диапазон длины текста поста  
- `MAX_TOTAL_RECORDS = 700000` — жёсткий глобальный лимит (не превысим)  
- `file_names = [tg.0.zip … tg.229.zip]` — список всех ZIP-файлов (можно ограничить для теста)

**Основная логика работы:**

1. **Загрузка предыдущего прогресса** (если запуск не первый):
   - `channel_top_posts` — топ-посты по каналам
   - `verified_channels_cache` — кэш проверенных категорий
   - `selected_posts_temp.jsonl` — уже отобранные посты
   - `progress.json` — список обработанных ZIP

2. **Функция фильтрации каналов** `is_channel_allowed()`  
   Проверяет, относится ли канал к одной из **разрешённых категорий** (из `filtered_categories.csv`).  
   Использует быстрый кэш, чтобы не проверять один и тот же канал тысячи раз.

3. **Умный отбор постов из середины** `select_mid_range_posts()`  
   Вместо топовых постов (которые часто — реклама / кликбейт) или самых непопулярных берём **середину** по просмотрам.  
   Это даёт более естественный, типичный контент канала.

4. **Основной цикл по ZIP-файлам**:
   - Скачивает каждый ZIP с Hugging Face (с кэшированием)
   - Распаковывает и читает построчно (JSONL внутри ZIP)
   - Для каждого поста:
     - Проверяет длину текста (500–1200 символов)
     - Проверяет категорию канала
     - Конвертирует просмотры (K → ×1000, M → ×1M)
     - Сохраняет в временный буфер `channel_top_posts[channel]`
   - Если буфер канала переполняется → оставляет только топ-20 (буфер ×4)
   - Периодически выводит статистику

5. **Промежуточный отбор после каждого ZIP**:
   - Из каждого канала берёт 5 постов из середины рейтинга
   - Дописывает их в `selected_posts_temp.jsonl` (режим append)
   - Сохраняет прогресс, статистику и кэш
   - Удаляет скачанный ZIP, чтобы не занимать место на диске

6. **Финальный этап** (после всех ZIP или достижения лимита):
   - Собирает все отобранные посты из временного файла
   - Ограничивает до `MAX_TOTAL_RECORDS`, если превысили
   - Сохраняет результат в **несколько форматов**:
     - `selected_posts.csv` — табличный вид (удобно для анализа)
     - `selected_posts.jsonl` — построчно (для больших данных)
     - `selected_posts_pretty.json` — красиво отформатированный JSON
     - `selected_posts.json` — компактный JSON
   - Создаёт сводную статистику `processing_summary.json`
   - Удаляет временный файл

**Результат выполнения**  
Готовый качественный датасет постов из нишевых Telegram-каналов:  
- только разрешённые категории  
- текст нормальной длины (500–1200 символов)  
- не топовые / не мусорные посты, а «средние по больнице»  
- до 700 000 записей (или меньше, если постов подходящего качества меньше)  
- несколько форматов на выбор + полная статистика обработки

Этот датасет идеально подходит для следующих шагов:  
- генерации описаний товаров по постам  
- разметки пар «пост — товар»  
- обучения моделей релевантности / ранжирования

**Важно:** ячейка устойчива к перезапускам — продолжает с того места, где остановилась.

In [ ]:
# Желаемое количество постов на канал
MAX_POSTS_PER_CHANNEL = 5

# Фильтр по длине текста
MIN_TEXT_LEN = 500
MAX_TEXT_LEN = 1200

# Глобальный лимит записей (не превысим)
MAX_TOTAL_RECORDS = 700000

# Список ZIP-файлов (0–112, но можно ограничить)
file_names = [f"tg.{i}.zip" for i in range(230)]
# file_names = file_names[:5]  # ← для теста

# ==============================================
# Вспомогательные структуры
# ==============================================

# channel → list of (views, text, obj) — храним только топ-5 по просмотрам
channel_top_posts = {}

# Кэш проверенных каналов (чтобы не проверять категорию каждый раз)
verified_channels_cache = {}

# Функция проверки, проходит ли канал по категории
def is_channel_allowed(channel_name):
    if channel_name in verified_channels_cache:
        return verified_channels_cache[channel_name]
    
    # Получаем категорию канала
    category = channel_to_category.get(channel_name)
    
    # Если категория не найдена или пустая — пропускаем
    if not category or pd.isna(category):
        verified_channels_cache[channel_name] = False
        return False
    
    # Проверяем, находится ли категория в разрешенных
    is_allowed = str(category) in allowed_categories
    verified_channels_cache[channel_name] = is_allowed
    
    return is_allowed

# Функция для выбора постов "из серединки"
def select_mid_range_posts(post_list, num_to_select=5):
    """
    Выбирает посты из середины отсортированного списка.
    Не берет ни самые популярные, ни самые непопулярные.
    """
    if not post_list or num_to_select <= 0:
        return []
    
    # Сортируем по просмотрам
    post_list.sort(key=lambda x: x[0], reverse=True)
    
    # Если постов меньше или равно нужному количеству - берем все
    if len(post_list) <= num_to_select:
        return [obj for _, _, obj in post_list]
    
    # Вычисляем позиции для выбора из середины
    total_posts = len(post_list)
    
    # Стратегия 1: берем равномерно из середины (пропускаем топ и низ)
    # Например, для 5 постов из 20: берем позиции 5, 8, 11, 14, 17
    if total_posts >= num_to_select * 4:
        # Много постов - берем равномерно из середины
        step = total_posts // (num_to_select + 1)
        start_idx = step
        indices = [start_idx + i*step for i in range(num_to_select)]
    else:
        # Мало постов - берем просто середину
        start_idx = max(1, (total_posts - num_to_select) // 2)
        indices = list(range(start_idx, start_idx + num_to_select))
    
    # Берем посты по вычисленным индексам
    selected = []
    for idx in indices:
        if idx < len(post_list):
            _, _, obj = post_list[idx]
            selected.append(obj)
    
    return selected

# Загрузка существующей статистики, если есть
if os.path.exists(stats_file):
    with open(stats_file, 'r', encoding='utf-8') as f:
        channel_top_posts = json.load(f)
    print(f"Загружено статистики по {len(channel_top_posts):,} каналам")

# Загрузка кэша проверенных каналов
cache_file = os.path.join(output_dir, "verified_channels_cache.json")
if os.path.exists(cache_file):
    with open(cache_file, 'r', encoding='utf-8') as f:
        verified_channels_cache = json.load(f)
    print(f"Загружен кэш для {len(verified_channels_cache)} каналов")

# Загрузка уже отобранных постов, если есть
selected_posts_count = 0
if os.path.exists(selected_posts_file):
    # Просто посчитаем количество строк
    with open(selected_posts_file, 'r', encoding='utf-8') as f:
        selected_posts_count = sum(1 for _ in f)
    print(f"Уже отобрано постов из предыдущих запусков: {selected_posts_count:,}")

# Прогресс: какие ZIP уже обработаны
processed_files = set()
if os.path.exists(progress_file):
    with open(progress_file, 'r') as pf:
        processed_files = set(json.load(pf))
    print(f"Уже обработано ZIP-файлов: {len(processed_files)}")

# ==============================================
# Основной цикл: чтение и отбор
# ==============================================

total_processed = 0
allowed_channels_count = 0

for filename in tqdm(file_names, desc="Обработка ZIP-файлов"):
    if filename in processed_files:
        print(f"{filename} уже обработан — пропуск")
        continue
    
    print(f"\n=== {filename} ===")
    
    try:
        file_path = hf_hub_download(
            repo_id=repo_id,
            filename=f"data/{filename}",
            repo_type="dataset",
            cache_dir=cache_dir
        )
        print(f"Скачано: {file_path}")
        
        # Открываем файл для добавления отобранных постов (режим 'a' - append)
        with open(selected_posts_file, 'a', encoding='utf-8') as spf:
            with zipfile.ZipFile(file_path, 'r') as z:
                inner_files = [f for f in z.namelist() if not f.endswith('/') and not f.endswith('.done')]
                print(f"Внутри ZIP файлов: {len(inner_files)}")
                
                for inner_path in tqdm(inner_files, desc="Внутренние файлы", leave=False):
                    with z.open(inner_path) as f:
                        for line in TextIOWrapper(f, encoding='utf-8', errors='ignore'):
                            line = line.strip()
                            if not line: continue
                            
                            total_processed += 1
                            
                            try:
                                obj = json.loads(line)
                                channel = obj.get('channel')
                                text = obj.get('text', '')
                                views = obj.get('views', 0)
                                
                                # Конвертируем views в число
                                if isinstance(views, str):
                                    if 'K' in views:
                                        views = float(views.replace('K', '').replace(',', '.')) * 1000
                                    elif 'M' in views:
                                        views = float(views.replace('M', '').replace(',', '.')) * 1000000
                                    else:
                                        try:
                                            views = float(views.replace(',', ''))
                                        except:
                                            views = 0
                                
                                if not isinstance(views, (int, float)):
                                    views = 0
                                
                                if not channel or not text:
                                    continue
                                
                                # Фильтр по длине
                                text_len = len(text)
                                if not (MIN_TEXT_LEN <= text_len <= MAX_TEXT_LEN):
                                    continue
                                
                                # Фильтр по категории канала
                                if not is_channel_allowed(channel):
                                    continue
                                
                                # Сохраняем пост для канала
                                if channel not in channel_top_posts:
                                    channel_top_posts[channel] = []
                                    allowed_channels_count += 1
                                
                                channel_top_posts[channel].append((views, text, obj))
                                
                                # Если у канала слишком много постов, оставляем только топ-N*2
                                if len(channel_top_posts[channel]) > MAX_POSTS_PER_CHANNEL * 4:  # увеличен буфер
                                    channel_top_posts[channel].sort(key=lambda x: x[0], reverse=True)
                                    channel_top_posts[channel] = channel_top_posts[channel][:MAX_POSTS_PER_CHANNEL * 4]
                                
                                # Периодический вывод статистики
                                if total_processed % 100000 == 0:
                                    # Подсчитываем текущее количество отобранных постов
                                    current_selected = sum(min(MAX_POSTS_PER_CHANNEL, len(posts)) for posts in channel_top_posts.values())
                                    print(f"Обработано: {total_processed:,} | Каналов с разрешенной категорией: {allowed_channels_count:,} | Потенциальных отобранных: {current_selected:,}")
                                
                                if len(channel_top_posts) > 0:
                                    # Подсчитываем примерное количество отобранных постов
                                    estimated_selected = sum(min(MAX_POSTS_PER_CHANNEL, len(posts)) for posts in channel_top_posts.values())
                                    if estimated_selected >= MAX_TOTAL_RECORDS:
                                        print(f"Достигнут глобальный лимит ({estimated_selected:,} записей) — прерываем")
                                        raise StopIteration
                                        
                            except json.JSONDecodeError:
                                continue
                            except Exception as e:
                                continue
        
        # После обработки каждого ZIP — проводим промежуточный отбор
        print("Промежуточный отбор постов из середины рейтинга...")
        
        # Отбираем посты из середины для каждого канала
        intermediate_selected = []
        for channel, post_list in channel_top_posts.items():
            if not post_list:
                continue
            
            # Используем новую функцию для отбора из середины
            selected_for_channel = select_mid_range_posts(post_list, MAX_POSTS_PER_CHANNEL)
            intermediate_selected.extend(selected_for_channel)
            
            # Останавливаемся при достижении лимита
            if len(intermediate_selected) >= MAX_TOTAL_RECORDS:
                intermediate_selected = intermediate_selected[:MAX_TOTAL_RECORDS]
                break
        
        # Сохраняем отобранные посты в файл
        with open(selected_posts_file, 'a', encoding='utf-8') as spf:
            for post in intermediate_selected:
                spf.write(json.dumps(post, ensure_ascii=False) + '\n')
        
        # Обновляем счетчик отобранных постов
        selected_posts_count += len(intermediate_selected)
        
        # Сохраняем прогресс
        processed_files.add(filename)
        with open(progress_file, 'w', encoding='utf-8') as pf:
            json.dump(list(processed_files), pf, indent=2)
        
        # Сохраняем промежуточную статистику топ-постов
        with open(stats_file, 'w', encoding='utf-8') as sf:
            json.dump(channel_top_posts, sf, ensure_ascii=False, indent=2)
        
        # Сохраняем кэш проверенных каналов
        with open(cache_file, 'w', encoding='utf-8') as cf:
            json.dump(verified_channels_cache, cf, ensure_ascii=False, indent=2)
        
        # Удаляем ZIP, чтобы не занимать место
        os.remove(file_path)
        print(f"ZIP удалён: {file_path}")
        
        print(f"Статистика после обработки {filename}:")
        print(f"  - Обработано записей: {total_processed:,}")
        print(f"  - Отобрано постов всего: {selected_posts_count:,}")
        print(f"  - Каналов с разрешенной категорией: {allowed_channels_count:,}")
        
        # Проверяем лимит
        if selected_posts_count >= MAX_TOTAL_RECORDS:
            print(f"Достигнут глобальный лимит в {MAX_TOTAL_RECORDS:,} записей — завершаем обработку")
            break
        
    except StopIteration:
        print("Обработка прервана по достижению лимита")
        break
    except Exception as e:
        print(f"Ошибка {filename}: {e}")
        import traceback
        traceback.print_exc()

# ==============================================
# Финальный отбор и сохранение результатов
# ==============================================

print("\nФинальный отбор постов...")
print(f"Всего каналов с разрешенной категорией: {len(channel_top_posts):,}")
print(f"Уже отобрано постов: {selected_posts_count:,}")

# Загружаем все отобранные посты из временного файла
selected_posts = []
if os.path.exists(selected_posts_file):
    with open(selected_posts_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                try:
                    selected_posts.append(json.loads(line))
                except:
                    continue

# Если нужно ограничить по лимиту (на случай если превысили)
if len(selected_posts) > MAX_TOTAL_RECORDS:
    print(f"Ограничиваем количество постов с {len(selected_posts):,} до {MAX_TOTAL_RECORDS:,}")
    selected_posts = selected_posts[:MAX_TOTAL_RECORDS]

print(f"\nФинальное количество постов: {len(selected_posts):,}")
print(f"Всего обработано записей: {total_processed:,}")

# ==============================================
# Сохранение финального результата
# ==============================================

if selected_posts:
    df_final = pd.DataFrame(selected_posts)
    
    # Сохраняем в CSV
    csv_path = os.path.join(output_dir, "selected_posts.csv")
    df_final.to_csv(csv_path, index=False, encoding='utf-8-sig')
    print(f"Финальный CSV: {csv_path} ({len(df_final):,} строк)")
    
    # Сохраняем в JSONL (одна строка = один пост)
    jsonl_path = os.path.join(output_dir, "selected_posts.jsonl")
    with open(jsonl_path, 'w', encoding='utf-8') as jf:
        for post in selected_posts:
            jf.write(json.dumps(post, ensure_ascii=False) + '\n')
    print(f"Финальный JSONL: {jsonl_path} ({len(selected_posts):,} записей)")
    
    # Сохраняем в ДВА JSON формата
    
    # 1. Красивый JSON с отступами (для чтения)
    # json_path_pretty = os.path.join(output_dir, "selected_posts_pretty.json")
    # with open(json_path_pretty, 'w', encoding='utf-8') as jf:
    #     json.dump(selected_posts, jf, ensure_ascii=False, indent=2)
    # print(f"Форматированный JSON: {json_path_pretty}")
    
    # 2. Обычный JSON без отступов (одна строка, компактный)
    # json_path_compact = os.path.join(output_dir, "selected_posts.json")
    # with open(json_path_compact, 'w', encoding='utf-8') as jf:
    #     json.dump(selected_posts, jf, ensure_ascii=False)
    # print(f"Компактный JSON: {json_path_compact} ({len(selected_posts):,} записей)")
    
    # Сохраняем статистику обработки
    stats_summary = {
        "total_selected_posts": len(selected_posts),
        "total_processed_records": total_processed,
        "total_allowed_channels": len(channel_top_posts),
        "max_posts_per_channel": MAX_POSTS_PER_CHANNEL,
        "min_text_length": MIN_TEXT_LEN,
        "max_text_length": MAX_TEXT_LEN,
        "max_total_records_limit": MAX_TOTAL_RECORDS,
        "allowed_categories_count": len(allowed_categories),
        "processed_zip_files": len(processed_files),
        "selection_strategy": "mid_range (not top, not bottom)",
        "estimated_memory_saved_MB": (total_processed - len(selected_posts)) * 0.5 / 1024  # примерная оценка
    }
    
    summary_path = os.path.join(output_dir, "processing_summary.json")
    with open(summary_path, 'w', encoding='utf-8') as sf:
        json.dump(stats_summary, sf, ensure_ascii=False, indent=2)
    print(f"Статистика обработки: {summary_path}")
    
    # Удаляем временный файл
    if os.path.exists(selected_posts_file):
        os.remove(selected_posts_file)
        print(f"Временный файл удалён: {selected_posts_file}")
    
else:
    print("Не удалось отобрать ни одного поста")

print("\nГотово!")

NameError: name 'os' is not defined

In [2]:
import json
import os
import pandas as pd
from tqdm import tqdm

# ==================== НАСТРОЙКИ ====================
input_jsonl   = "data/posts/ajtkulov/selected/selected_posts.jsonl"           # ваш исходный файл
channels_csv  = "data/posts/ajtkulov/meta/all_channels_clean_with_cols.csv"             # файл с категориями
output_jsonl  = "data/posts/ajtkulov/selected/selected_posts_with_category.jsonl"  # куда писать результат
progress_file = "data/posts/ajtkulov/selected/last_processed_line.txt"        # файл с номером последней строки

# Поля, которые должны быть в channels_csv
CHANNEL_COL = "link"       # столбец с username канала (без @)
CATEGORY_COL = "category"  # столбец с категорией

# ==================== ПОДГОТОВКА ====================
# Загружаем словарь канал → категория (один раз)
print("Загружаем словарь категорий...")
channels_df = pd.read_csv(channels_csv, usecols=[CHANNEL_COL, CATEGORY_COL])
channel_to_category = dict(zip(channels_df[CHANNEL_COL], channels_df[CATEGORY_COL]))
print(f"Загружено {len(channel_to_category):,} каналов с категориями")

# Определяем, с какой строки продолжать
start_line = 0
if os.path.exists(progress_file):
    with open(progress_file, "r") as f:
        try:
            start_line = int(f.read().strip())
            print(f"Возобновление с строки {start_line + 1}")
        except:
            print("Не удалось прочитать прогресс → начинаем с начала")
            start_line = 0

# Подсчитываем общее количество строк (для tqdm)
total_lines = sum(1 for _ in open(input_jsonl, encoding="utf-8"))
print(f"Всего строк в исходном файле: {total_lines:,}")

# Открываем выходной файл в режиме append
with open(output_jsonl, "a", encoding="utf-8") as out_f:
    # Пропускаем уже обработанные строки
    with open(input_jsonl, "r", encoding="utf-8") as in_f:
        for i, line in enumerate(tqdm(in_f, total=total_lines, desc="Обработка строк", initial=start_line)):
            if i < start_line:
                continue  # пропускаем уже сделанное

            line = line.strip()
            if not line:
                continue

            try:
                post = json.loads(line)
                channel = post.get("channel", "")

                # Ищем категорию
                category = channel_to_category.get(channel, "UNKNOWN")
                post["category"] = category

                # Записываем сразу в выходной файл
                out_f.write(json.dumps(post, ensure_ascii=False) + "\n")

                # Сохраняем прогресс после каждой строки
                with open(progress_file, "w") as pf:
                    pf.write(str(i))

            except json.JSONDecodeError:
                print(f"Ошибка JSON в строке {i+1} → пропуск")
                continue
            except Exception as e:
                print(f"Неизвестная ошибка в строке {i+1}: {e}")
                continue

print("\nОбработка завершена!")
print(f"Новый файл: {output_jsonl}")
print(f"Прогресс сохранён в: {progress_file}")
print("Чтобы начать заново — удалите файл прогресса.")

Загружаем словарь категорий...
Загружено 380,468 каналов с категориями
Всего строк в исходном файле: 700,000


Обработка строк: 100%|██████████| 700000/700000 [03:19<00:00, 3517.23it/s]


Обработка завершена!
Новый файл: data/posts/ajtkulov/selected/selected_posts_with_category.jsonl
Прогресс сохранён в: data/posts/ajtkulov/selected/last_processed_line.txt
Чтобы начать заново — удалите файл прогресса.


In [ ]:
import pandas as pd
import json
import os
from tqdm import tqdm

# ==================== НАСТРОЙКИ ====================
input_file = "data/posts/ajtkulov/selected/selected_posts_with_category.jsonl"
output_file = "data/posts/ajtkulov/selected/selected10k.jsonl"
N_TOTAL = 10000          # ← сколько постов всего хотим в итоге
# Если хотите ровно по N постов на категорию — укажите здесь, иначе равномерно распределяет
# N_PER_CATEGORY = 200   # ← раскомментируйте, если хотите фиксированное число на категорию
CATEGORY_COL = "category"

# ==================== ЗАГРУЗКА ====================
print("Загружаем файл постов...")
df = pd.read_json(input_file, lines=True)
print(f"Всего постов в исходном файле: {len(df):,}")

if CATEGORY_COL not in df.columns:
    raise ValueError(f"Столбец '{CATEGORY_COL}' не найден. Доступные столбцы: {df.columns.tolist()}")

# Очистка и подсчёт
df[CATEGORY_COL] = df[CATEGORY_COL].fillna("Без категории").astype(str)
category_counts = df[CATEGORY_COL].value_counts()
print(f"Уникальных категорий: {len(category_counts)}")
print("\nТоп-10 категорий по количеству постов:")
print(category_counts.head(10))

# ==================== СТРАТИФИЦИРОВАННЫЙ ОТБОР ====================
print("\nОтбираем посты равномерно по категориям...")
selected = []

# Вариант 1: равномерное распределение (по пропорции)
if 'N_PER_CATEGORY' not in globals() or N_PER_CATEGORY is None:
    # Делим N_TOTAL пропорционально количеству постов в категории
    total_posts = len(df)
    for cat, count in tqdm(category_counts.items(), desc="Отбор по категориям"):
        if count == 0:
            continue
        # Сколько взять от этой категории
        n_take = max(1, round(count / total_posts * N_TOTAL))
        # Берём случайные посты
        cat_posts = df[df[CATEGORY_COL] == cat].sample(n=n_take, random_state=42)
        selected.append(cat_posts)
else:
    # Вариант 2: фиксированное число постов на категорию (если раскомментировали N_PER_CATEGORY)
    n_take = N_PER_CATEGORY
    for cat in tqdm(category_counts.index, desc="Отбор по категориям"):
        cat_posts = df[df[CATEGORY_COL] == cat]
        if len(cat_posts) == 0:
            continue
        sampled = cat_posts.sample(n=min(n_take, len(cat_posts)), random_state=42)
        selected.append(sampled)

# Объединяем
if selected:
    df_selected = pd.concat(selected, ignore_index=True)
    # Перемешиваем, чтобы не было блоков по категориям
    df_selected = df_selected.sample(frac=1, random_state=42).reset_index(drop=True)
    
    print(f"\nОтобрано постов: {len(df_selected):,}")
    print(f"Уникальных категорий в выборке: {df_selected[CATEGORY_COL].nunique()}")
    
    # Статистика по категориям в новой выборке
    new_counts = df_selected[CATEGORY_COL].value_counts()
    print("\nРаспределение в новой выборке (топ-10):")
    print(new_counts.head(10))
    
    # ────────────────────────────────────────────────
    # ФИКС ОШИБКИ: преобразуем Timestamp в строку
    if 'date' in df_selected.columns and pd.api.types.is_datetime64_any_dtype(df_selected['date']):
        print("Конвертируем столбец 'date' в строку для JSON...")
        df_selected['date'] = df_selected['date'].astype(str)
    # ────────────────────────────────────────────────
    
    # Сохраняем
    with open(output_file, 'w', encoding='utf-8') as f:
        for _, row in tqdm(df_selected.iterrows(), total=len(df_selected), desc="Сохранение"):
            f.write(json.dumps(row.to_dict(), ensure_ascii=False) + '\n')
    
    print(f"\nГотово! Новый файл сохранён: {output_file}")
    print(f"Размер: {len(df_selected):,} постов")
else:
    print("Не удалось отобрать ни одного поста — проверьте данные.")

Загружаем файл постов...
Всего постов в исходном файле: 700,000
Уникальных категорий: 124

Топ-10 категорий по количеству постов:
category
Блоги                 125923
Криптовалюты           47220
Игры                   46386
Мода и красота         43054
Юмор и развлечение     28034
Экономика              26413
Продажи                26245
Технологии             25817
Психология             24307
Еда и кулинария        20208
Name: count, dtype: int64

Отбираем посты равномерно по категориям...


Отбор по категориям: 124it [00:06, 18.68it/s]



Отобрано постов: 10,029
Уникальных категорий в выборке: 124

Распределение в новой выборке (топ-10):
category
Блоги                 1799
Криптовалюты           675
Игры                   663
Мода и красота         615
Юмор и развлечение     400
Экономика              377
Продажи                375
Технологии             369
Психология             347
Еда и кулинария        289
Name: count, dtype: int64


Сохранение:   0%|          | 0/10029 [00:00<?, ?it/s]


TypeError: Object of type Timestamp is not JSON serializable